In [41]:
from anomalib.models import Patchcore
from anomalib.data import Folder
from anomalib.engine import Engine
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import cv2
from tqdm import tqdm
import os
from PIL import Image
from torch.utils.data import Dataset
from anomalib.deploy import export


In [14]:
input_dir  = "../data/tilda/train/good"          
output_dir = "../data/tilda/train/good_clean" 

os.makedirs(output_dir, exist_ok=True)

print("Nettoyage et standardisation de data/train/good ...")
counter = 0

for filename in tqdm(os.listdir(input_dir)):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp', '.tiff')):
        src_path = os.path.join(input_dir, filename)
        
        # Lecture directe en gris
        img = cv2.imread(src_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
            
        # CLAHE + resize 256×256
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced = clahe.apply(img)
        final = cv2.resize(enhanced, (256, 256))
        
        # Nom propre et extension fixe
        new_name = f"good_{counter:06d}.png"
        dst_path = os.path.join(output_dir, new_name)
        
        cv2.imwrite(dst_path, final)
        counter += 1

print(f"Terminé ! {counter} images normales propres dans data/train/good_clean")

Nettoyage et standardisation de data/train/good ...


100%|████████████████████████████████████████████████████████████████████████████████| 771/771 [00:08<00:00, 91.85it/s]

Terminé ! 770 images normales propres dans data/train/good_clean


In [13]:
datamodule = Folder(
    name="textile_anomaly",
    root="../data/tilda",
    normal_dir="train/good_clean",
    abnormal_dir="test",  # même dossier, pas de labels réels
    num_workers=0,
    train_batch_size=1,
    eval_batch_size=1,
)

In [48]:
model = Patchcore(
    backbone="resnet18",
    layers=["layer2"],      
    pre_trained=True,
    num_neighbors=3,          
    coreset_sampling_ratio=0.01,  
)


In [15]:
engine = Engine(
    accelerator="cpu",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=1,
)

In [16]:
engine.fit(model=model, datamodule=datamodule)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\core\optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0 | pre_processor  | PreProcessor   | 0      | train
1 | post_processor | PostProcessor  | 0      | train
2 | evaluator      | Evaluator      | 0      | train
3 | model          | PatchcoreModel | 683 K  | train
----------------------------------------------------------
683 K     Trainable params
0         Non-trainable params
683 K     Total params
2.732     Total estimated model params size (MB)
19        Modules in train mode
47        Modules in eval mode
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be 

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…



ing Coreset Indices.:   0%|                                                             | 0/6307 [00:00<?, ?it/s]

ing Coreset Indices.:   0%|                                                     | 1/6307 [00:00<46:37,  2.25it/s]

ing Coreset Indices.:   0%|                                                     | 2/6307 [00:00<53:08,  1.98it/s]

ing Coreset Indices.:   0%|                                                     | 3/6307 [00:01<47:22,  2.22it/s]

ing Coreset Indices.:   0%|                                                     | 4/6307 [00:01<46:11,  2.27it/s]

ing Coreset Indices.:   0%|                                                     | 5/6307 [00:02<47:23,  2.22it/s]

ing Coreset Indices.:   0%|                                                     | 6/6307 [00:02<44:42,  2.35it/s]

ing Coreset Indices.:   0%|                                                     | 7/6307 [00:02<37:13,  2.82it/s]

ing Coreset Indices.:   0%|                                                   

In [17]:
engine.test(model=model, datamodule=datamodule)


The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric AUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.8108288645744324     │
│       image_F1Score       │    0.7032967209815979     │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 0.8108288645744324, 'image_F1Score': 0.7032967209815979}]

In [25]:
metrics1 = engine.test(model=model, datamodule=datamodule)


The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor


Testing: |                                                                                       | 0/? [00:00<…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.8108288645744324     │
│       image_F1Score       │    0.7032967209815979     │
└───────────────────────────┴───────────────────────────┘

In [18]:
torch.save(model.state_dict(), "../models/patchcore1.pt")
print("Modèle PatchCore1 sauvegardé ")


Modèle PatchCore1 sauvegardé 


### 1ère amélioration du modèle 

In [57]:
datamodule2 = Folder(
    name="textile_anomaly_model2",
    root="../data/tilda",
    normal_dir="train/good_clean",
    abnormal_dir="val", 
    num_workers=0,
    train_batch_size=1,
    eval_batch_size=1,
)

In [58]:
model2 = Patchcore(
    backbone="resnet18",
    layers=["layer2", "layer3"],   #  amélioration
    pre_trained=True,
    num_neighbors=3,
    coreset_sampling_ratio=0.01,
)

In [59]:
engine2 = Engine(
    accelerator="cpu",
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=1,
)

In [60]:
engine2.fit(model=model2, datamodule=datamodule2)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\core\optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0 | pre_processor  | PreProcessor   | 0      | train
1 | post_processor | PostProcessor  | 0      | train
2 | evaluator      | Evaluator      | 0      | train
3 | model          | PatchcoreModel | 2.8 M  | train
----------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)
19        Modules in train mode
69        Modules in eval mode
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be 

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…



ing Coreset Indices.:   0%|                                                             | 0/6307 [00:00<?, ?it/s]

ing Coreset Indices.:   0%|                                                   | 1/6307 [00:00<1:02:32,  1.68it/s]

ing Coreset Indices.:   0%|                                                     | 2/6307 [00:00<34:56,  3.01it/s]

ing Coreset Indices.:   0%|                                                     | 3/6307 [00:01<33:31,  3.13it/s]

ing Coreset Indices.:   0%|                                                     | 4/6307 [00:01<25:50,  4.06it/s]

ing Coreset Indices.:   0%|                                                     | 5/6307 [00:01<25:45,  4.08it/s]

ing Coreset Indices.:   0%|                                                     | 6/6307 [00:01<21:51,  4.80it/s]

ing Coreset Indices.:   0%|                                                     | 7/6307 [00:01<20:33,  5.11it/s]

ing Coreset Indices.:   0%|                                                   

In [42]:
engine2.test(model=model2, datamodule=datamodule)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric AUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9656226634979248     │
│       image_F1Score       │    0.8805969953536987     │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 0.9656226634979248, 'image_F1Score': 0.8805969953536987}]

In [66]:
engine2.test(model=model2, datamodule=datamodule2)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
C:\Users\Nouhayla\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |                                                                                       | 0/? [00:00<…

C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric AUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
C:\Users\Nouhayla\anaconda3\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9460842609405518     │
│       image_F1Score       │    0.8403361439704895     │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 0.9460842609405518, 'image_F1Score': 0.8403361439704895}]

In [67]:
model2.post_processor.image_threshold


tensor(12.0534)